### Data Processing

5. You must build tilted portfolios based on the $R^2 , \sigma^{2}_{i} , \beta^{2}_{i} \sigma^{2}_{M} , \sigma^{2}_{ei} , \alpha_i , r_i$


6. To get a tilted portfolio you should sort all the stocks in ascending or descending order depending on the feature or combination/ration of them.

7. If you have 100 stocks, when sorted you must select the upper and lower
quantile, for example 10% or 20%.

8. To build a portfolio with the selected stocks you can simply set an equally weighted portfolio. For example, in case of 10 stocks, 10% is invested in each asset.

9. To get the returns of the portfolio, simply calculate the weighted average of
stock returns involved.

10. The portfolios must be rebalanced weekly until the end of the sample.

Imports

In [1]:
import numpy as np
import pandas as pd

import time
import datetime as dt # for defining dates

import matplotlib.pyplot as plt # plotting
import matplotlib.dates as mdates # styling dates
%matplotlib inline

from functions import *

In [2]:
import os
import math
import statistics as st

Costants

In [3]:
# folder used to store stock data
folder = "../data/"

# size of the window used for rolling regression
sample_size = 180

Read Data

In [4]:
df_ES50 = pd.read_csv('../data/raw/ES50.csv', header=1)
df_ES50 = df_ES50.iloc[:, : 52]
df_ES50 = df_ES50.drop([0])
df_ES50 = df_ES50.rename(columns={"Symbols": "Date", "^STOXX50E": "EuroStoxx50"})
df_ES50['Date'] = pd.to_datetime(df_ES50['Date'])
df_ES50.head()

,Date,EuroStoxx50,ADS.DE,ADYEN.AS,AD.AS,AI.PA,AIR.PA,ALV.DE,ABI.BR,ASML.AS,...,SAF.PA,SAN.PA,SAP.DE,SU.PA,SIE.DE,STLA.MI,TTE.PA,DG.PA,VOW.DE,VNA.DE
1,2016-01-04,3164.760010,78.004707,NaN,15.247333,63.063118,53.968327,111.405846,90.266624,74.629242,...,57.882877,59.244190,63.260578,41.909256,68.110123,4.946972,27.490080,47.195293,117.650673,22.053017
2,2016-01-05,3178.010010,77.737396,NaN,15.321729,62.687923,55.059048,111.762909,91.510849,75.216354,...,57.576126,59.946838,64.022430,42.097519,68.540154,5.077475,27.239086,47.456890,111.036285,22.200811
3,2016-01-06,3139.320068,76.837410,NaN,15.223837,61.812473,54.331902,112.119972,90.306778,73.464317,...,57.167126,59.137260,64.578133,41.352646,68.269386,4.813435,26.977922,47.178940,110.737564,21.741453
4,2016-01-07,3084.679932,75.643364,NaN,15.145526,60.386761,53.241184,109.799019,88.781578,71.749557,...,56.051670,58.495708,63.941761,40.501366,66.963402,4.655616,26.323318,46.639381,105.446068,21.334026
5,2016-01-08,3033.469971,74.360207,NaN,15.176851,59.467541,52.106125,108.406448,86.333267,68.720772,...,55.410286,57.029305,63.681831,40.644611,66.501534,4.464415,25.336323,46.484043,106.726273,20.994507


In [5]:
ES50_log_returns = pd.read_csv('../data/ES50_log-ret.csv')
ES50_log_returns = ES50_log_returns.drop(['Unnamed: 0'], axis=1)
ES50_log_returns.head(7)

,Date,EuroStoxx50,ADS.DE,ADYEN.AS,AD.AS,AI.PA,AIR.PA,ALV.DE,ABI.BR,ASML.AS,...,SAF.PA,SAN.PA,SAP.DE,SU.PA,SIE.DE,STLA.MI,TTE.PA,DG.PA,VOW.DE,VNA.DE
0,2016-01-05,0.004178,-0.003433,NaN,0.004867,-0.005967,0.020009,0.003200,0.013690,0.007836,...,-0.005314,0.011790,0.011971,0.004482,0.006294,0.026038,-0.009172,0.005528,-0.057863,0.006679
1,2016-01-06,-0.012249,-0.011645,NaN,-0.006410,-0.014064,-0.013295,0.003190,-0.013245,-0.023569,...,-0.007129,-0.013597,0.008642,-0.017852,-0.003958,-0.053403,-0.009634,-0.005874,-0.002694,-0.020908
2,2016-01-07,-0.017558,-0.015662,NaN,-0.005157,-0.023335,-0.020279,-0.020918,-0.017033,-0.023618,...,-0.019705,-0.010908,-0.009903,-0.020801,-0.019315,-0.033337,-0.024564,-0.011502,-0.048963,-0.018917
3,2016-01-08,-0.016741,-0.017109,NaN,0.002066,-0.015339,-0.021550,-0.012764,-0.027964,-0.043130,...,-0.011509,-0.025388,-0.004073,0.003531,-0.006921,-0.041936,-0.038216,-0.003336,0.012068,-0.016042
4,2016-01-11,-0.001973,0.021458,NaN,0.008222,-0.012060,-0.011123,0.003945,0.019794,0.026101,...,-0.016918,-0.013754,0.007711,-0.019317,-0.006487,-0.004087,0.000000,0.021402,0.013503,0.003987
5,2016-01-12,0.012203,0.012008,NaN,0.001534,0.027813,0.015200,0.012390,-0.007319,0.003956,...,0.012210,0.009459,0.036343,0.006243,0.005768,0.027602,-0.002681,0.016222,0.020305,0.012429
6,2016-01-13,0.002724,-0.006394,NaN,0.009409,0.004750,0.008271,0.001619,0.004124,-0.008458,...,0.004876,0.004429,-0.007028,0.001529,-0.003842,-0.013369,0.015979,0.007425,0.004628,-0.004126


In [6]:
rank_df = pd.read_csv('../data/ES50_parameters.csv')
rank_df = rank_df.drop(['Unnamed: 0'], axis=1)
rank_df.head()

,tickers,r2,beta,alpha,alpha_significance,absolute_returns,specific_risk,systematic_risk,total_risk
0,ADS.DE,0.177656,0.412880,0.003112,0.002831,0.559717,0.013751,0.042638,0.056389
1,AD.AS,0.319903,0.473584,0.000715,0.371868,0.128262,0.010689,0.056098,0.066787
2,AI.PA,0.536992,0.719739,0.000329,0.671121,0.058461,0.010346,0.129569,0.139915
3,AIR.PA,0.587508,0.947977,-0.000391,0.671284,-0.071300,0.012296,0.224775,0.237071
4,ALV.DE,0.813180,1.058462,-0.000226,0.700797,-0.041755,0.007854,0.280222,0.288076


Ranking

- Max $R^2$
- Max _Absolute Returns_
- Min _Total Risk_
- Min _Systematic Risk_
- Max or Min _Specific Risk_
- Max and Significant $\alpha$

Max $R^2$

In [7]:
max_r2 = rank_df.sort_values(by='r2', ascending=False).head(10)
max_r2_selected = max_r2['tickers'].tolist()

print(max_r2_selected)
max_r2

['ALV.DE', 'CS.PA', 'BNP.PA', 'INGA.AS', 'SAN.MC', 'BAS.DE', 'BBVA.MC', 'BMW.DE', 'SIE.DE', 'MUV2.DE']


,tickers,r2,beta,alpha,alpha_significance,absolute_returns,specific_risk,systematic_risk,total_risk
4,ALV.DE,0.813180,1.058462,-0.000226,0.700797,-0.041755,0.007854,0.280222,0.288076
7,CS.PA,0.788997,1.495205,-0.000472,0.598192,-0.086560,0.011970,0.559182,0.571152
13,BNP.PA,0.782196,1.523465,0.000256,0.783756,0.044451,0.012445,0.580520,0.592965
23,INGA.AS,0.764207,1.501316,0.000353,0.714772,0.062031,0.012910,0.563762,0.576672
11,SAN.MC,0.755679,1.786274,0.000213,0.856557,0.036456,0.015723,0.798083,0.813806
8,BAS.DE,0.736871,0.870956,0.000719,0.234398,0.128438,0.008057,0.189734,0.197791
10,BBVA.MC,0.735194,1.559311,-0.000344,0.750962,-0.063557,0.014487,0.608159,0.622646
12,BMW.DE,0.717737,1.143192,-0.000466,0.575273,-0.084990,0.011098,0.326881,0.337979
36,SIE.DE,0.713783,0.919542,0.001441,0.033833,0.258409,0.009014,0.211493,0.220507
29,MUV2.DE,0.698328,0.849834,0.000036,0.955419,0.005638,0.008647,0.180642,0.189289


Max _Absolute Returns_

In [8]:
max_absolute_returns = rank_df.sort_values(by='absolute_returns', ascending=False).head(10)
max_absolute_returns_selected = max_absolute_returns['tickers'].tolist()

Min _Total Risk_

In [9]:
min_total_risk = rank_df.sort_values(by='total_risk').head(10)
min_total_risk_selected = min_total_risk['tickers'].tolist()

Min _Systematic Risk_

In [10]:
min_systematic_risk = rank_df.sort_values(by='systematic_risk').head(10)
min_systematic_risk_selected = min_systematic_risk['tickers'].tolist()

Max _Specific Risk_

In [11]:
max_specific_risk = rank_df.sort_values(by='specific_risk', ascending=False).head(10)
max_specific_risk_selected = max_specific_risk['tickers'].tolist()

Max $\beta$

In [12]:
max_beta = rank_df.sort_values(by='beta', ascending=False).head(10)
max_beta_selected = max_beta['tickers'].tolist()

Min $\beta$

In [13]:
min_beta = rank_df.sort_values(by='beta').head(10)
min_beta_selected = min_beta['tickers'].tolist()

Max and Significant $\alpha$

In [14]:
max_significant_alpha = rank_df[rank_df['alpha_significance'] < 0.05]
max_significant_alpha.sort_values(by='alpha', ascending=False).head(10)
max_significant_alpha_selected = max_significant_alpha['tickers'].tolist()

In [15]:
max_significant_alpha

# only 2 titles, not enough

,tickers,r2,beta,alpha,alpha_significance,absolute_returns,specific_risk,systematic_risk,total_risk
0,ADS.DE,0.177656,0.412880,0.003112,0.002831,0.559717,0.013751,0.042638,0.056389
36,SIE.DE,0.713783,0.919542,0.001441,0.033833,0.258409,0.009014,0.211493,0.220507


Selected Stocks

In [16]:
print(max_r2_selected)
print(max_absolute_returns_selected)
print(min_total_risk_selected)
print(min_systematic_risk_selected)
print(max_specific_risk_selected)
print(max_beta_selected)

['ALV.DE', 'CS.PA', 'BNP.PA', 'INGA.AS', 'SAN.MC', 'BAS.DE', 'BBVA.MC', 'BMW.DE', 'SIE.DE', 'MUV2.DE']
['ADS.DE', 'SIE.DE', 'IFX.DE', 'VNA.DE', 'KER.PA', 'SU.PA', 'DG.PA', 'ASML.AS', 'RMS.PA', 'SAP.DE']
['LIN.DE', 'ADS.DE', 'AD.AS', 'VNA.DE', 'BN.PA', 'RI.PA', 'RMS.PA', 'OR.PA', 'IBE.MC', 'ABI.BR']
['LIN.DE', 'ADS.DE', 'AD.AS', 'VNA.DE', 'BN.PA', 'RI.PA', 'RMS.PA', 'OR.PA', 'IBE.MC', 'ABI.BR']
['VOW.DE', 'SAN.MC', 'BBVA.MC', 'IFX.DE', 'ADS.DE', 'SAF.PA', 'VNA.DE', 'INGA.AS', 'ASML.AS', 'DB1.DE']
['SAN.MC', 'BBVA.MC', 'BNP.PA', 'INGA.AS', 'CS.PA', 'VOW.DE', 'BMW.DE', 'SU.PA', 'ALV.DE', 'MBG.DE']


Weekly Log Returns

In [17]:
print(df_ES50.shape)
print(df_ES50.columns)

(1712, 52)
Index(['Date', 'EuroStoxx50', 'ADS.DE', 'ADYEN.AS', 'AD.AS', 'AI.PA', 'AIR.PA',
       'ALV.DE', 'ABI.BR', 'ASML.AS', 'CS.PA', 'BAS.DE', 'BAYN.DE', 'BBVA.MC',
       'SAN.MC', 'BMW.DE', 'BNP.PA', 'CRG.IR', 'BN.PA', 'DB1.DE', 'DPW.DE',
       'DTE.DE', 'ENEL.MI', 'ENI.MI', 'EL.PA', 'FLTR.IR', 'RMS.PA', 'IBE.MC',
       'ITX.MC', 'IFX.DE', 'INGA.AS', 'ISP.MI', 'KER.PA', 'KNEBV.HE', 'OR.PA',
       'LIN.DE', 'MC.PA', 'MBG.DE', 'MUV2.DE', 'RI.PA', 'PHIA.AS', 'PRX.AS',
       'SAF.PA', 'SAN.PA', 'SAP.DE', 'SU.PA', 'SIE.DE', 'STLA.MI', 'TTE.PA',
       'DG.PA', 'VOW.DE', 'VNA.DE'],
      dtype='object')


In [18]:
df_weekly_log_returns = pd.DataFrame()

j = 0
for col in df_ES50.columns[1:53]:
  new_row = []
  date_row = []
  for i in range(sample_size + 1, len(df_ES50) - 7, 7):
    date_row.append(df_ES50['Date'][i])
    new_row.append(np.log(df_ES50[col][i+7]) - np.log(df_ES50[col][i]))

  df_weekly_log_returns['Date'] = date_row
  df_weekly_log_returns[col] = new_row

In [19]:
print(df_weekly_log_returns.shape)
df_weekly_log_returns.head(5)

(218, 52)


,Date,EuroStoxx50,ADS.DE,ADYEN.AS,AD.AS,AI.PA,AIR.PA,ALV.DE,ABI.BR,ASML.AS,...,SAF.PA,SAN.PA,SAP.DE,SU.PA,SIE.DE,STLA.MI,TTE.PA,DG.PA,VOW.DE,VNA.DE
0,2016-09-14,0.022481,0.044159,NaN,0.004545,-0.015440,0.014036,0.039953,0.064424,0.075588,...,0.019563,-0.019817,0.038589,0.063099,0.035949,0.008719,0.024875,0.034660,-0.005795,0.024016
1,2016-09-23,-0.000927,0.014720,NaN,0.004525,0.007132,0.019143,-0.027084,-0.000861,0.006968,...,0.021899,0.007858,-0.001962,-0.014076,-0.007087,-0.000869,0.028154,-0.003792,0.011939,-0.030749
2,2016-10-04,-0.018140,-0.027046,NaN,-0.048434,-0.050053,-0.045314,0.019367,-0.014751,-0.077358,...,-0.067350,-0.012104,-0.035863,-0.020670,-0.019632,-0.026410,0.006704,-0.042236,-0.000383,-0.052550
3,2016-10-13,0.039162,0.012970,NaN,0.038645,0.005184,0.044949,0.046839,0.003055,0.061922,...,0.030425,0.015143,0.035372,0.037442,0.034224,0.050445,0.021653,0.031954,0.034629,0.036051
4,2016-10-24,-0.037439,-0.057348,NaN,-0.026993,-0.014431,-0.041712,-0.032555,-0.105603,-0.028079,...,-0.049430,0.037580,-0.049466,-0.049020,-0.055236,0.048831,-0.044964,-0.043915,-0.045024,-0.038745


In [20]:
df_weekly_log_returns.to_csv('../data/ES50_weekly_log-ret.csv')

Portfolio Building

In [21]:
# index
ndx_pfl = df_weekly_log_returns[['EuroStoxx50']]

In [22]:
# portfolio building
max_r2_pfl = df_weekly_log_returns[max_r2_selected]
max_absolute_returns_pfl = df_weekly_log_returns[max_absolute_returns_selected]
min_total_risk_pfl = df_weekly_log_returns[min_total_risk_selected]
min_systematic_risk_pfl = df_weekly_log_returns[min_systematic_risk_selected]
max_specific_risk_pfl = df_weekly_log_returns[max_specific_risk_selected]
max_beta_pfl = df_weekly_log_returns[max_beta_selected]

Annual Returns and Volatility

In [23]:
# index annual return
ndx_pfl_mean = ndx_pfl.mean()
ndx_pfl_mean = ndx_pfl_mean.to_list()

ndx_pfl_annual_return = sum(ndx_pfl_mean) / ndx_pfl.shape[1] * 52
print("Annualized Returns: " + str(ndx_pfl_annual_return))

# index annual standard deviation
ndx_pfl_std = ndx_pfl.std()
ndx_pfl_std = ndx_pfl_std.to_list()

ndx_pfl_annual_volatility = sum(ndx_pfl_std) / ndx_pfl.shape[1] * math.sqrt(52)
print("Annualized Volatility: " + str(ndx_pfl_annual_volatility))

Annualized Returns: 0.04695922502961186
Annualized Volatility: 0.24431768573753215


Max $R^2$

In [24]:
# portfolio annual return
max_r2_pfl_mean = max_r2_pfl.mean()
max_r2_pfl_mean = max_r2_pfl_mean.to_list()

max_r2_pfl_annual_return = sum(max_r2_pfl_mean) / max_r2_pfl.shape[1] * 52
print("Annualized Returns: " + str(max_r2_pfl_annual_return))

# portfolio annual standard deviation
max_r2_pfl_std = max_r2_pfl.std()
max_r2_pfl_std = max_r2_pfl_std.to_list()

max_r2_pfl_annual_volatility = sum(max_r2_pfl_std) / max_r2_pfl.shape[1] * math.sqrt(52)
print("Annualized Volatility: " + str(max_r2_pfl_annual_volatility))

Annualized Returns: 0.06885104575327805
Annualized Volatility: 0.36625113738072


Max _Absolute Returns_

In [25]:
# portfolio annual return
max_absolute_returns_pfl_mean = max_absolute_returns_pfl.mean()
max_absolute_returns_pfl_mean = max_absolute_returns_pfl_mean.to_list()

max_absolute_returns_pfl_annual_return = sum(max_absolute_returns_pfl_mean) / max_absolute_returns_pfl.shape[1] * 52
print("Annualized Returns: " + str(max_absolute_returns_pfl_annual_return))

# portfolio annual standard deviation
max_absolute_returns_pfl_std = max_absolute_returns_pfl.std()
max_absolute_returns_pfl_std = max_absolute_returns_pfl_std.to_list()

max_absolute_returns_pfl_annual_volatility = sum(max_absolute_returns_pfl_std) / max_absolute_returns_pfl.shape[1] * math.sqrt(52)
print("Annualized Volatility: " + str(max_absolute_returns_pfl_annual_volatility))

Annualized Returns: 0.17438184140185328
Annualized Volatility: 0.33956447496425135


Min _Total Risk_

In [26]:
# portfolio annual return
min_total_risk_pfl_mean = min_total_risk_pfl.mean()
min_total_risk_pfl_mean = min_total_risk_pfl_mean.to_list()

min_total_risk_pfl_annual_return = sum(min_total_risk_pfl_mean) / min_total_risk_pfl.shape[1] * 52
print("Annualized Returns: " + str(min_total_risk_pfl_annual_return))

# portfolio annual standard deviation
min_total_risk_pfl_std = min_total_risk_pfl.std()
min_total_risk_pfl_std = min_total_risk_pfl_std.to_list()

min_total_risk_pfl_annual_volatility = sum(min_total_risk_pfl_std) / min_total_risk_pfl.shape[1] * math.sqrt(52)
print("Annualized Volatility: " + str(min_total_risk_pfl_annual_volatility))

Annualized Returns: 0.13246648816836157
Annualized Volatility: 0.29231395306451263


Min _Systematic Risk_

In [27]:
# portfolio annual return
min_systematic_risk_pfl_mean = min_systematic_risk_pfl.mean()
min_systematic_risk_pfl_mean = min_systematic_risk_pfl_mean.to_list()

min_systematic_risk_pfl_annual_return = sum(min_systematic_risk_pfl_mean) / min_systematic_risk_pfl.shape[1] * 52
print("Annualized Returns: " + str(min_systematic_risk_pfl_annual_return))

# portfolio annual standard deviation
min_systematic_risk_pfl_std = min_systematic_risk_pfl.std()
min_systematic_risk_pfl_std = min_systematic_risk_pfl_std.to_list()

min_systematic_risk_pfl_annual_volatility = sum(min_systematic_risk_pfl_std) / min_systematic_risk_pfl.shape[1] * math.sqrt(52)
print("Annualized Volatility: " + str(min_systematic_risk_pfl_annual_volatility))

Annualized Returns: 0.13246648816836157
Annualized Volatility: 0.29231395306451263


Max _Specific Risk_

In [28]:
# portfolio annual return
max_specific_risk_pfl_mean = max_specific_risk_pfl.mean()
max_specific_risk_pfl_mean = max_specific_risk_pfl_mean.to_list()

max_specific_risk_pfl_annual_return = sum(max_specific_risk_pfl_mean) / max_specific_risk_pfl.shape[1] * 52
print("Annualized Returns: " + str(max_specific_risk_pfl_annual_return))

# portfolio annual standard deviation
max_specific_risk_pfl_std = max_specific_risk_pfl.std()
max_specific_risk_pfl_std = max_specific_risk_pfl_std.to_list()

max_specific_risk_pfl_annual_volatility = sum(max_specific_risk_pfl_std) / max_specific_risk_pfl.shape[1] * math.sqrt(52)
print("Annualized Volatility: " + str(max_specific_risk_pfl_annual_volatility))

Annualized Returns: 0.11691006800503526
Annualized Volatility: 0.3868485676940349


Max $\beta$

In [29]:
# portfolio annual return
max_beta_pfl_mean = max_beta_pfl.mean()
max_beta_pfl_mean = max_beta_pfl_mean.to_list()

max_beta_pfl_annual_return = sum(max_beta_pfl_mean) / max_beta_pfl.shape[1] * 52
print("Annualized Returns: " + str(max_beta_pfl_annual_return))

# portfolio annual standard deviation
max_beta_pfl_std = max_beta_pfl.std()
max_beta_pfl_std = max_beta_pfl_std.to_list()

max_beta_pfl_annual_volatility = sum(max_beta_pfl_std) / max_beta_pfl.shape[1] * math.sqrt(52)
print("Annualized Volatility: " + str(max_beta_pfl_annual_volatility))

Annualized Returns: 0.09248991288342147
Annualized Volatility: 0.3840030134800512


Final Statistics - Table

In [30]:
final_stats = pd.DataFrame(columns=['Portfolio', 'Annualized Returns', 'Annualized Volatility'])

In [31]:
pfl_list = ['NDX ES50', 'Max R2', 'Max Absolute Returns', 'Min Total Risk', 'Min Systematic Risk', 'Max Specific Risk', 'Max Beta']

In [32]:
final_stats.loc[len(final_stats)] = [pfl_list[0], ndx_pfl_annual_return, ndx_pfl_annual_volatility]
final_stats.loc[len(final_stats)] = [pfl_list[1], max_r2_pfl_annual_return, max_r2_pfl_annual_volatility]
final_stats.loc[len(final_stats)] = [pfl_list[2], max_absolute_returns_pfl_annual_return, max_absolute_returns_pfl_annual_volatility]
final_stats.loc[len(final_stats)] = [pfl_list[3], min_total_risk_pfl_annual_return, min_total_risk_pfl_annual_volatility]
final_stats.loc[len(final_stats)] = [pfl_list[4], min_systematic_risk_pfl_annual_return, min_systematic_risk_pfl_annual_volatility]
final_stats.loc[len(final_stats)] = [pfl_list[5], max_specific_risk_pfl_annual_return, max_specific_risk_pfl_annual_volatility]
final_stats.loc[len(final_stats)] = [pfl_list[6], max_beta_pfl_annual_return, max_beta_pfl_annual_volatility]

In [33]:
final_stats

,Portfolio,Annualized Returns,Annualized Volatility
0,NDX ES50,0.046959,0.244318
1,Max R2,0.068851,0.366251
2,Max Absolute Returns,0.174382,0.339564
3,Min Total Risk,0.132466,0.292314
4,Min Systematic Risk,0.132466,0.292314
5,Max Specific Risk,0.116910,0.386849
6,Max Beta,0.092490,0.384003


In [34]:
final_stats.to_csv('../data/ES50_final_stats.csv')